# VaR Model

- 609 Project
- Author: Yu Chunxue
- Date: 29/02/2024

<a id="top" style="font-size: 24px; color: black;"> 💪🏻 CONTENT</a>

## [1.  Parametric VaR Model ](#p1)

## [2.  Monte Carlo VaR Model ](#p2)

## [3.   Historical VaR Model ](#p3)

In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy.stats import norm

#### 1.Define the VaR parameters:
-  Significance level: 95%
-  Risk horizon: 1-day 
-  Reporting currency

#### 2. Identify the portfolio:
-  A payer (pay fixed leg) SOFR swap
-  $1 million in each of the four stocks: AAPL, MSFT, F (Ford Motor) and BAC (Bank of America)

#### 3.Identify the risk factors of the portfolio:
- An interest rate curve
- When conducting VaR calculations or other risk measurements, fluctuations in SOFR can impact the market value of swap transactions.

#### 4.Model the joint distribution of the risk factors over the risk horizon
- Parametric VaR
- Monte Carlo VaR
- Historical VaR

#### 5.Build the distribution of the portfolio P&L over the risk horizon


#### <span style="color:red;">6. Calculate the VaR</span>

## 0. Clean Data

### 0.1 Swap

In [2]:
sofr_curve = pd.read_excel("hist_data.xlsm", sheet_name="SofrCurve")

In [3]:
sofr_curve_transposed = sofr_curve.T
sofr_curve_transposed = sofr_curve_transposed.drop(sofr_curve_transposed.index[0])

In [4]:
sofr_curve_transposed.columns = sofr_curve_transposed.iloc[0]
sofr_curve_transposed = sofr_curve_transposed.drop(sofr_curve_transposed.index[0])

In [5]:
sofr_curve_transposed.index = pd.to_datetime(sofr_curve_transposed.index, format="%Y-%m-%d")

In [6]:
sofr_curve_transposed

T,0.002778,0.083333,0.166667,0.250000,0.500000,0.750000,1.000000,2.000000,3.000000,4.000000,...,15.000000,16.000000,17.000000,18.000000,19.000000,20.000000,25.000000,30.000000,35.000000,40.000000
2022-10-31,0.039191,0.038721,0.03867,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.03235,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.04697,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.03601,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.0391,0.040852,0.044884,0.04658,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.03935,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.0475,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.03738,0.037152,0.03687,0.036534,0.034424,0.032558,0.030723,0.028933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.053105,0.053084,0.053202,0.053424,0.05369,0.053293,0.052503,0.048399,0.045999,0.04485,...,0.044093,0.044056,0.043972,0.043841,0.043663,0.043437,0.041874,0.040238,0.038438,0.036493
2023-10-25,0.052981,0.053057,0.053259,0.05349,0.053749,0.053393,0.052653,0.048791,0.046595,0.045594,...,0.045289,0.045268,0.045198,0.045078,0.044906,0.044683,0.043076,0.041358,0.039564,0.037641
2023-10-26,0.053047,0.053089,0.053218,0.053414,0.053567,0.053089,0.052243,0.048044,0.045645,0.044538,...,0.044269,0.044244,0.044171,0.044049,0.043882,0.043668,0.042182,0.040589,0.03876,0.036764
2023-10-27,0.052989,0.05304,0.053185,0.053368,0.053486,0.052991,0.052115,0.047758,0.045284,0.0442,...,0.04446,0.044461,0.044411,0.044309,0.044157,0.043955,0.042508,0.040985,0.039167,0.037148


### 0.2 Stocks

In [7]:
hist_data_multiple_sheets = pd.read_excel("hist_data.xlsm", sheet_name=["AAPL", "MSFT", "F", "BAC"])

merged_data = None

for sheet_name, data in hist_data_multiple_sheets.items():
    if merged_data is None:
        merged_data = data
    else:
        merged_data = pd.merge(merged_data, data, on="Date", how="outer", suffixes=("", "_" + sheet_name))

In [8]:
merged_data["Date"] = pd.to_datetime(merged_data["Date"])

merged_data.set_index("Date", inplace=True)

In [9]:
merged_data.columns.values[0] = "Adj Close_AAPL"

In [10]:
merged_data

,Adj Close_AAPL,Adj Close_MSFT,Adj Close_F,Adj Close_BAC
Date,,,,
2022-10-31,152.041122,229.443207,11.974380,34.748581
2022-11-01,149.373917,225.529037,12.001248,34.902847
2022-11-02,143.801514,217.552444,11.696740,34.796783
2022-11-03,137.703613,211.770157,11.875863,34.603954
2022-11-04,137.435455,218.827515,12.099767,35.471703
...,...,...,...,...
2023-10-24,172.991058,329.860504,11.215128,25.268324
2023-10-25,170.657135,339.979980,11.362824,25.347691
2023-10-26,166.458023,327.225861,11.175742,25.913177


In [11]:
df1 = sofr_curve_transposed
df2 = merged_data

##   <a id = "p1">1. Para VaR Model </a>   [back](#top)

### 1.1 Swap

In [12]:
# Discount factor
#df_discountfactor = pd.DataFrame(1 / (1 + df1)).apply(lambda row: row ** pd.to_numeric(df1.columns), axis=1)
df1 = df1.apply(pd.to_numeric, errors='coerce')
df_discountfactor = pd.DataFrame(np.exp(-df1)).apply(lambda row: row ** pd.to_numeric(df1.columns), axis=1)

In [13]:
# Fixed leg - 2023/10/30
PV_fixedLeg = sum(1e8 * 0.042 * value for value in df_discountfactor.loc['2023-10-30', 1:10])
PV_fixedLeg

33168363.251927644

In [14]:
# Floating leg - 2023/10/30
PV_floatingLeg = 1e8 * (1 - df_discountfactor.loc['2023-10-30', 10])
PV_floatingLeg

35611265.25177399

In [15]:
# PV - 2023/10/30
PV_swap = PV_floatingLeg - PV_fixedLeg
PV_swap

2442901.9998463467

In [16]:
# Calculate a - weight: a^k
## every 0.1% change of SOFA make how many change of PV

a_list = pd.DataFrame(index=['a'])
newPV_fixedLegs =[]
for column_name in df1.columns:
    if 1.0 <= float(column_name) <= 10.0:
        newdf1 = df1.copy()
        newdf1.loc['2023-10-30', column_name] += 0.0001
        newdf_discountfactor = pd.DataFrame(np.exp(-newdf1)).apply(lambda row: row ** pd.to_numeric(newdf1.columns), axis=1)
        
        newPV_fixedLeg = sum(1e8 * 0.042 * value for value in newdf_discountfactor.loc['2023-10-30', 1:10])
        newPV_fixedLegs.append(newPV_fixedLeg)
        
        newPV_floatingLeg =  1e8 * (1 - newdf_discountfactor.loc['2023-10-30', 10])
        a_list[column_name] = (newPV_floatingLeg - newPV_fixedLeg)
        
    else: 
        a_list[column_name] = (PV_floatingLeg - PV_fixedLeg) 

a_list = (a_list - PV_swap) * 1e4 

In [17]:
# Get absolute change: delta x^k
L1 = df1.diff()

### 1.2 Stockes

In [18]:
# Calculate daily log returns
L2 = pd.DataFrame(index = df2.index)
for col in ["Adj Close_AAPL", "Adj Close_MSFT", "Adj Close_F", "Adj Close_BAC"]:
    L2[col + '_return'] = np.log(df2[col]/df2[col].shift(1))

In [19]:
# Weight - a^k = 1e6
for column_name in L2.columns:
    a_list[column_name] = 1e6

In [20]:
PV_stocks = (np.exp(L2.sum()) * 1e6).sum()

In [21]:
PV_stocks

4121111.7358001657

### 1.3 Combine 2 type of assets

In [22]:
L2.index = L1.index

In [23]:
L = pd.concat([L1,L2],axis = 1).dropna()

### 1.4 Calculate VaR

In [24]:
# Mean and Var
mu = L.mean()
mu_p = a_list.values @ mu
mu_p = mu_p[0]
mu_p

20787.92138927857

In [25]:
covariance = L.cov()
covariance_p = a_list @ covariance @ a_list.T
covariance_p = covariance_p.iloc[0,0]
covariance_p

332277750399.5879

In [26]:
# Parametric VaR
## 95% Confidence Level
VaR_parametric_95 = abs(scipy.stats.norm.ppf(0.05, mu_p, np.sqrt(covariance_p)))
VaR_parametric_95

927363.908506733

##   <a id = "p2">2. MC VaR Model </a>   [back](#top)

### 2.1 Risk-based approach

In [27]:
# Stimulate new change
num_simulations = 10000 
newL_mean = np.random.multivariate_normal(L.mean(), L.cov(), num_simulations)

In [28]:
new_mu_p = []
for newL_mean_factor in newL_mean:
    result = a_list.values @ newL_mean_factor
    new_mu_p.append(result)

In [29]:
new_mu_p.sort()

In [30]:
VaR_MC_risk_based_95 = abs(new_mu_p[499])[0]
VaR_MC_risk_based_95

931605.5062427421

### 2.2 Full revaluation approach

In [31]:
portfolio_value = PV_swap + PV_stocks

In [32]:
data = pd.concat([df1.loc['2023-10-30',:],df2.loc['2023-10-30',:]],axis = 0)

In [33]:
new_data = []
for newL_mean_factor in newL_mean:
    result = data + newL_mean_factor
    new_data.append(result)

#### swap

In [34]:
new_discountfactor = []
for scenario in new_data:
    result = np.exp(-scenario).values[:30] ** df1.columns.values
    new_discountfactor.append(result)

In [35]:
new_fixedLeg = []
for df in new_discountfactor:
    result = sum(1e8 * 0.042 * value for value in df[6:16])
    new_fixedLeg.append(result)

In [36]:
new_floatingLeg = []
for df in new_discountfactor:
    result = 1e8 * (1 - df[15])
    new_floatingLeg.append(result)

In [37]:
newPV_swap = np.array(new_floatingLeg) - np.array(new_fixedLeg)

#### stocks

In [38]:
newPV_stocks =[]
for scenario in new_data:
    result = (scenario[-4:]/ data[-4:] * 1e6).sum()
    newPV_stocks.append(result)

#### Combine

In [39]:
new_portfolio_value = newPV_swap + newPV_stocks

In [40]:
VaR_MC_full_revaluation = new_portfolio_value - portfolio_value
VaR_MC_full_revaluation.sort()

In [41]:
VaR_MC_full_revaluation = abs(VaR_MC_full_revaluation[499])
VaR_MC_full_revaluation

1067899.518168386

In [43]:
result_df = pd.DataFrame(columns=['VAR'])
result_df.loc['para', 'VAR'] = VaR_parametric_95
result_df.loc['MC_fullre', 'VAR'] = VaR_MC_full_revaluation
result_df.loc['MC_riskbased', 'VAR'] = VaR_MC_risk_based_95

In [44]:
result_df

,VAR
para,927363.908507
MC_fullre,1067899.518168
MC_riskbased,931605.506243
